# Pyspark_Streaming_Basic

In [ ]:
http://spark.apache.org/docs/latest/streaming-kafka-0-8-integration.html

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


import json
import re
import numpy as np
import pandas

In [2]:
sc = SparkContext()

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.history.fs.cleaner.interval', u'7d'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  u'http://hdp-all-spark-new.c.kao-project-v1.internal:8088/proxy/application_1541652388947_0024'),
 (u'spark.shuffle.io.serverThreads', u'128'),
 (u'spark.driver.appUIAddress',
  u'http://hdp-all-spark-new.c.kao-project-v1.internal:4041'),
 (u'spark.yarn.historyServer.address',
  u'hdp-all-spark-new.c.kao-project-v1.internal:18081'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/l

In [5]:
print sc.getConf().toDebugString()

spark.app.id=application_1541652388947_0024
spark.app.name=pyspark-shell
spark.driver.appUIAddress=http://hdp-all-spark-new.c.kao-project-v1.internal:4041
spark.driver.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.driver.host=hdp-all-spark-new.c.kao-project-v1.internal
spark.driver.port=37030
spark.eventLog.dir=hdfs:///spark2-history/
spark.eventLog.enabled=true
spark.executor.extraJavaOptions=-XX:+UseNUMA
spark.executor.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.executor.id=driver
spark.executorEnv.PYTHONPATH=/usr/hdp/3.0.0.0-1634/spark2/python:/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip
spark.history.fs.cleaner.enabled=true
spark.history.fs.cleaner.interval=7d
spark.history.fs.cleaner.maxAge=90d
spark.history.fs.logDirectory=hdfs:///spark2-history/
spark.histo

In [6]:
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

In [ ]:
#Use to stop each lab
ssc.stop(stopSparkContext=False)

In [8]:
#Create streaming context and DStream
ssc = StreamingContext(sc, 5)
kafkaStream = KafkaUtils.createStream(ssc, "localhost:2181","group1", {"twitter":1})
text_ds = kafkaStream.map(lambda x: x[1])

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o199.createStream

In [ ]:
#Lab1 - simple streaming 
#show amount of tweets per batch interval

text_ds.count().map(lambda x:'Tweets in this batch: %d' % x).pprint()

ssc.start()

In [ ]:
#Lab2 - simple streaming
#show sample tweet

text_ds.pprint()
ssc.start()

In [ ]:
#Lab3 - foreachRDD
#show sample tweet using dataframe 

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    #     schema = StructType([StructField("text", StringType(), nullable = False)])
    #     text_df = text_rdd.toDF(schema = schema)
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    text_df.show(n=5, truncate=False)

text_ds.foreachRDD(process)

ssc.start()

In [ ]:
#Lab4 - foreachRDD + transformation(ML)
#tokenize tweet to word list

from pyspark.ml.feature import Tokenizer

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    tokens_df.show(n=5, truncate=False)

text_ds.foreachRDD(process)

ssc.start()

In [ ]:
#Lab5 - foreachRDD + transformation(ML)
#tokenize tweet to word list
#remove stop word

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")

    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = remover.transform(tokens_df)
    tokens_df.show(n=5, truncate=False)


text_ds.foreachRDD(process)

ssc.start()

In [ ]:
#Lab6 - foreachRDD + transformation(ML)
#tokenize tweet to word list
#remove stop word
#flatmap word list to word

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    token_df.show(n=5, truncate=False)

text_ds.foreachRDD(process)

ssc.start()

In [ ]:
#Lab7 - foreachRDD + transformation(ML)
#tokenize tweet to word list
#remove stop word
#flatmap word list to word
#word count

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    
    token_count_df = (token_df
                          .groupBy('word')
                          .count()
                          .sort(desc("count"))
                     )
    token_count_df.show(n=5, truncate=False)

text_ds.foreachRDD(process)

ssc.start()

In [ ]:
#Lab8 - transform + transformation(ML)
#tokenize tweet to word list
#remove stop word
#flatmap word list to word
#word count

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    print("========= %s =========" % str(time))
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    
    token_count_df = (token_df
                          .groupBy('word')
                          .count()
                          .sort(desc("count"))
                     )

    return token_count_df.rdd


token_count_ds = text_ds.transform(process)
token_count_ds.pprint()

ssc.start()

In [ ]:
#Lab9 - window
#show amount of tweets per batch interval
#show amount of tweets per window

ssc.checkpoint("checkpoints")

text_ds.count().map(lambda x:'Tweets in this batch: %d' % x).pprint()
text_window_ds = text_ds.window(20)
text_window_ds.count().map(lambda x:'Tweets in last 20 seconds : %d' % x).pprint()

ssc.start()

In [ ]:
#Lab10 - window - reduceByKeyAndWindow
#From word,count
#To word,count

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    
    token_count_df = (token_df
                          .groupBy('word')
                          .count()
                          .sort(desc("count"))
                     )
    return token_count_df.rdd



def show(time, token_count_rdd):
    print("========= %s =========" % str(time))
    spark = getSparkSessionInstance(token_count_rdd.context.getConf())
    
    row_rdd = token_count_rdd.map(lambda (word,count): Row(word=word,count=count))
    token_df = spark.createDataFrame(row_rdd)
    
    (token_df
          .sort(desc("count"))
          .show(n=5, truncate=False)
    )

    
    
ssc.checkpoint("checkpoints")

#Row(word,count)
token_count_ds = text_ds.transform(process)
token_count_ds.foreachRDD(show)

## Window - Reduce last 30 seconds of data, every 5 seconds
addFunc = lambda x, y: x + y
invAddFunc = lambda x, y: x - y
token_count_window_ds = token_count_ds.reduceByKeyAndWindow(addFunc, invAddFunc, 30, 5)
token_count_window_ds.foreachRDD(show)

ssc.start()

In [ ]:
#Lab11 - Window - countByValueAndWindow (1)
#word

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    
    return token_df.rdd


    
ssc.checkpoint("checkpoints")

token_ds = text_ds.transform(process)
token_ds.pprint()

ssc.start()


In [ ]:
#Lab12 - Window - countByValueAndWindow (1)
#From word
#To word,count

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    
    return token_df.rdd



def show(time, token_count_rdd):
    print("========= %s =========" % str(time))
    spark = getSparkSessionInstance(token_count_rdd.context.getConf())
    
    row_rdd = token_count_rdd.map(lambda (word,count): Row(word=word,count=count))
    token_df = spark.createDataFrame(row_rdd)
    
    print row_rdd.first()
    
    token_df.printSchema()
    
    (token_df
          .sort(desc("count"))
          .show(n=5, truncate=False)
    )



ssc.checkpoint("checkpoints")

token_ds = text_ds.transform(process)


## Window - Reduce last 30 seconds of data, every 5 seconds
token_count_window_ds = token_ds.countByValueAndWindow(30, 5)
token_count_window_ds.foreachRDD(show)

ssc.start()

In [ ]:
#Stateful
#From word,count
#To word,count

from pyspark.ml.feature import Tokenizer, StopWordsRemover

def process(time, text_rdd):
    
    spark = getSparkSessionInstance(text_rdd.context.getConf())
    
    row_rdd = text_rdd.map(lambda p: Row(text=p))
    text_df = spark.createDataFrame(row_rdd)
    
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    tokens_df = tokenizer.transform(text_df).select("words")
    
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    tokens_df = (remover
                     .transform(tokens_df)
                     .drop('words')
                     .withColumnRenamed("filtered","words")
                    )
    token_df = (tokens_df
                    .select(explode(col("words")).alias("word"))
                    .where(length(col("word")) >= 3)
               )
    token_count_df = (token_df
                          .groupBy('word')
                          .count()
                          .sort(desc("count"))
                     )
    return token_count_df.rdd.map(tuple)


def show(time, token_count_rdd):
    print("========= %s =========" % str(time))
    spark = getSparkSessionInstance(token_count_rdd.context.getConf())
    
    row_rdd = token_count_rdd.map(lambda (word,count): Row(word=word,count=count))
    token_df = spark.createDataFrame(row_rdd)
    
    (token_df
          .sort(desc("count"))
          .show(n=5, truncate=False)
    )



#Seq[Int],Option[Int]
def update_value_func(new_values, state_value):
    if state_value is None:
        state_value = 0
    sum_value = np.sum(new_values) + state_value
    return float(sum_value)




ssc.checkpoint("checkpoints")
token_ds = text_ds.transform(process)


## Stateful
token_stateful_ds = token_ds.updateStateByKey(update_value_func)
token_stateful_ds.foreachRDD(show)

ssc.start()